In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.spatial.distance import cosine

In [2]:
dial_test = pd.read_parquet("./Data/dial_test.parquet")
dial_train = pd.read_parquet("./Data/dial_train.parquet")

In [3]:
train_target = pd.read_parquet('./Data/train_target.parquet')
test_target = pd.read_parquet('./Data/test_target_b.parquet')

In [4]:
dial_test['mon'] = dial_test.event_time.dt.month
dial_test['year'] = dial_test.event_time.dt.year
dial_train['mon'] = dial_test.event_time.dt.month
dial_train['year'] = dial_test.event_time.dt.year

In [5]:
test_target['mon'] = pd.to_datetime(test_target['mon']) 
test_target['mon'] = test_target['mon'] - pd.DateOffset(months=1)
test_target['year'] = test_target['mon'].dt.year
test_target['mon'] = test_target['mon'].dt.month


train_target['mon'] = pd.to_datetime(train_target['mon']) 
train_target['mon'] = train_target['mon'] - pd.DateOffset(months=1)
train_target['year'] = train_target['mon'].dt.year
train_target['mon'] = train_target['mon'].dt.month


In [6]:
dial_test = pd.merge(dial_test, test_target, on=['client_id', 'mon','year'], how='inner')
dial_train = pd.merge(dial_train, train_target, on=['client_id', 'mon','year'], how='inner')

In [7]:
def aggregate_embeddings(embeddings):
    return np.mean(embeddings, axis=0)

In [8]:
dial_test = pd.merge(dial_test.groupby(['client_id','mon'])['embedding'].apply(lambda x: aggregate_embeddings(np.vstack(x))).reset_index(), test_target, on=['client_id', 'mon'], how='inner')
dial_train = pd.merge(dial_train.groupby(['client_id','mon'])['embedding'].apply(lambda x: aggregate_embeddings(np.vstack(x))).reset_index(), train_target, on=['client_id', 'mon'], how='inner')

In [9]:
df = pd.concat([dial_test, dial_train])

In [10]:
target = ['target_1', 'target_2','target_3','target_4']

In [11]:
from scipy.spatial.distance import cosine, euclidean, cityblock
from tqdm.auto import tqdm
#список функций
distances = [cosine, euclidean, cityblock]

#функция подсчёта расстояния 
def compute_cosine_distance(row):
    return distance(given_embedding, row)

#применяем каждую функцию к эмбеддингу
for fun in tqdm(distances):
    distance = fun
    s = [f'{repr(distance).split()[1]}_pos', f'{repr(distance).split()[1]}_neg']
    for i in target:
        for j in [0,1]:        
            given_embedding = df.groupby(i)['embedding'].apply(lambda x: aggregate_embeddings(np.vstack(x))).reset_index()['embedding'][j]
            dial_test[f'{s[j]}_{i}'] = dial_test['embedding'].apply(compute_cosine_distance)
            dial_train[f'{s[j]}_{i}'] = dial_train['embedding'].apply(compute_cosine_distance)
        
dial_train    

  0%|          | 0/3 [00:00<?, ?it/s]

,client_id,mon,embedding,target_1,target_2,target_3,target_4,year,cosine_pos_target_1,cosine_neg_target_1,...,euclidean_pos_target_4,euclidean_neg_target_4,cityblock_pos_target_1,cityblock_neg_target_1,cityblock_pos_target_2,cityblock_neg_target_2,cityblock_pos_target_3,cityblock_neg_target_3,cityblock_pos_target_4,cityblock_neg_target_4
0,000006265d27d1166ed67506682be7380007a5bead4362...,2.0,"[0.3184107, -0.051863734, 0.44436622, -0.35518...",0,0,0,0,2022,0.019426,0.021404,...,3.045701,3.190551,62.878746,70.785393,63.149143,55.356853,63.175552,61.156364,63.054199,67.233253
1,000006265d27d1166ed67506682be7380007a5bead4362...,3.0,"[0.39509884, -0.25605047, 0.53794205, -0.59390...",0,0,0,0,2022,0.017542,0.020109,...,3.814221,3.997308,95.921646,104.338173,96.223969,86.798340,96.253029,93.982178,96.118118,100.639328
2,00019eed9ed218e2c59d668c46d3ff841fee03ab225b20...,10.0,"[0.4400319, -0.30076867, 0.5862988, -0.5225228...",0,0,0,0,2022,0.024129,0.027725,...,5.334719,5.531397,137.334824,146.784821,137.670715,127.062744,137.700653,135.254395,137.550156,142.737350
3,00021a8c0dd1ffba57d5d690e1f97f11e7a771d1cef5e7...,3.0,"[0.2666735, -0.3521267, 0.58935714, -0.2870635...",0,0,0,0,2022,0.024136,0.026537,...,3.841040,4.002924,78.787140,88.211517,79.119164,69.433372,79.149193,76.735428,78.999428,84.179314
4,00021a8c0dd1ffba57d5d690e1f97f11e7a771d1cef5e7...,10.0,"[0.38029987, -0.20136076, 0.53209764, -0.39702...",1,0,0,0,2022,0.016718,0.018428,...,3.023305,3.180732,78.457703,85.651970,78.717697,70.278107,78.746132,76.616653,78.624504,82.569954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165562,fffdfd12873c30eac72e4b40b928c947e0e6c773c567e5...,8.0,"[0.5413299, -0.36577246, 0.49462152, -0.572501...",0,0,0,0,2022,0.031198,0.034335,...,5.076559,5.263182,117.756500,127.741943,118.116302,106.652580,118.149414,115.470825,117.986496,123.538406
165563,fffdfd12873c30eac72e4b40b928c947e0e6c773c567e5...,9.0,"[0.04685535, -0.13879849, 0.38878888, -0.04042...",0,0,0,0,2022,0.057514,0.055777,...,3.421290,3.331213,69.076202,67.462425,68.980072,73.482452,68.970459,69.811142,69.014343,67.900726
165564,fffdfd12873c30eac72e4b40b928c947e0e6c773c567e5...,10.0,"[0.4457826, -0.15867385, 0.55251867, -0.395958...",0,0,0,0,2022,0.012629,0.015126,...,3.584777,3.777292,87.206116,96.983124,87.558006,76.470200,87.590195,84.990601,87.431625,92.845139
165565,fffdfd12873c30eac72e4b40b928c947e0e6c773c567e5...,12.0,"[0.2780109, -0.15078264, 0.4774778, -0.2813794...",0,0,0,0,2022,0.014966,0.016712,...,2.908111,3.067571,59.480331,68.907486,59.814072,49.554375,59.845161,57.361252,59.694206,64.890953


In [12]:
dial_test.to_parquet(f'./Data/test_emb_dial_v{5}.parquet', engine='pyarrow', index=False)
dial_train.to_parquet(f'./Data/train_emb_dial_v{5}.parquet', engine='pyarrow', index=False)

# Препроцессинг эмбеддингов ptls

In [13]:
emb_trx = pd.read_parquet("./Data/trx_emb_select_1_v2.parquet")
emb_geo = pd.read_parquet("./Data/geo_emb_select_1_v2.parquet")

In [14]:
emb_trx

,client_id,report_next_end,emb_trx
19,00fe1344ab467920aeae2008c2e351d6ecc6d09c0192ae...,2022-11-30,"[1.0, -0.21202722, 0.31116307, 1.0, 1.0, 0.189..."
30,0163fc28a8ec289d03340d56ea13e975b4cf4d0cea6141...,2022-11-30,"[-1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -0.043..."
47,0218864e5f62e44ae86c490de5c3c0822ec3a79ae11476...,2022-11-30,"[-0.0026537764, -0.21202722, 0.31116307, 1.0, ..."
54,02745013e5cf8b2e1896c4f77e89bc3ca952057dd46486...,2022-11-30,"[1.0, -0.21202722, -1.0, -1.0, -0.03291097, 0...."
77,03c4d714e95fcdbfa6b77acf234011d23f51b639a1b697...,2022-11-30,"[-0.0026537764, -0.21202722, -1.0, 1.0, -0.032..."
...,...,...,...
1080734,e26f303b215d861753db9598bdc31022122406469ac3bc...,2022-12-30,"[1.0, -0.21202722, 1.0, 1.0, 0.9999963, -1.0, ..."
1080916,eb7f7ce03df433da1ba312a359d9ec1e071eb966cc6aea...,2022-12-30,"[1.0, 0.9999888, -1.0, 1.0, 1.0, 0.18988582, 1..."
1080928,ebb38c0ca39d3a5aefe95e166704eff9c59f7dcb7c3f5c...,2022-12-30,"[1.0, 1.0, -1.0, 0.9999967, -0.03291097, 0.998..."
1081073,f29acdd92ea6054e8e2ea19eb761e30ff04135aff45203...,2022-12-30,"[-0.0026537764, -0.21202722, -1.0, -0.1718208,..."


In [15]:
emb_trx['event_time'] = emb_trx.report_next_end.map(lambda x: x - relativedelta(months=1))
emb_trx['mon'] = emb_trx['event_time'].dt.month
emb_trx['year'] = emb_trx['event_time'].dt.year

emb_geo['event_time'] = emb_geo.report_next_end.map(lambda x: x - relativedelta(months=1))
emb_geo['mon'] = emb_geo['event_time'].dt.month
emb_geo['year'] = emb_geo['event_time'].dt.year

In [16]:
df_targ = pd.concat([train_target, test_target])

In [17]:
df_trx = pd.merge(df_targ, emb_trx, on=['client_id', 'mon','year'], how='inner')
df_geo = pd.merge(df_targ, emb_geo, on=['client_id', 'mon','year'], how='inner')

In [18]:
df_trx

,mon,target_1,target_2,target_3,target_4,client_id,year,report_next_end,emb_trx,event_time
0,4,0,0,0,0,1d55174bce3ef488233380aca206ca2fb51661b1c46495...,2022,2022-05-31,"[-1.0, -0.99988127, -1.0, -0.1718208, -0.03291...",2022-04-30
1,5,1,0,0,0,1d5d052f87d6bd22a30b5df160fffdcdd11014feffe75c...,2022,2022-06-30,"[-1.0, -0.21202722, 0.31116307, -0.1718208, -0...",2022-05-30
2,5,0,0,0,0,1d68b588164639d64879b33da867818ea7401b6eb7caff...,2022,2022-06-30,"[-1.0, -0.21202722, -1.0, -0.1718208, -1.0, 1....",2022-05-30
3,7,0,0,0,0,1d817e82e1cc594e32e10217b716faa3ad079261975097...,2022,2022-08-31,"[1.0, 1.0, -1.0, 1.0, -0.03291097, -1.0, -0.48...",2022-07-31
4,5,1,0,0,0,1daf5849cf68892983c584c8ee97b8329307584a834e79...,2022,2022-06-30,"[1.0, -0.98695296, 1.0, 1.0, 1.0, -1.0, -0.481...",2022-05-30
...,...,...,...,...,...,...,...,...,...,...
406137,6,0,0,0,0,ff0c295f5b534ca6c2d47ad91d0fbe97644f42de9df873...,2022,2022-07-31,"[1.0, 1.0, -1.0, 1.0, -0.03291097, 1.0, 1.0, 1...",2022-06-30
406138,8,0,0,0,0,ff4a98bcab383f2117cbb91c0d9eb95a5008de0367b1dd...,2022,2022-09-30,"[-1.0, -0.21202722, -1.0, -0.1718208, -0.03291...",2022-08-30
406139,8,0,0,0,0,ff7cc72f879ec6b041fa7f23ff91f23362c5c9b0ad9b3a...,2022,2022-09-30,"[-1.0, -0.21202722, 1.0, -0.1718208, -0.032910...",2022-08-30
406140,3,0,0,0,0,ffc09c3decd0f53763e4737accf686bade81f1fc86da94...,2022,2022-04-30,"[-1.0, -0.21202722, 1.0, -1.0, -1.0, -1.0, -1....",2022-03-30


In [19]:
from scipy.spatial.distance import cosine, euclidean, cityblock
from tqdm.auto import tqdm
#список функций
distances = [cosine, euclidean, cityblock]
col = 'emb_trx' # наименование колонки с эбедингами
target = ['target_1', 'target_2','target_3','target_4']


#применяем каждую функцию к эмбеддингу
def fun(df, col):   
    for fun in tqdm(distances):
        distance = fun
        s = [f'{repr(distance).split()[1]}_pos', f'{repr(distance).split()[1]}_neg']
        for i in target:
            for j in [0,1]:        
                given_embedding = df.groupby(i)[col].apply(lambda x: aggregate_embeddings(np.vstack(x))).reset_index()[col][j]
                #функция подсчёта расстояния 
                def compute_cosine_distance(row):
                    return distance(given_embedding, row)
                df[f'{s[j]}_{i}'] = df[col].apply(compute_cosine_distance)
    return df


In [20]:
fun(df_trx, col).to_parquet(f'./Data/trx_emb_select_1_v3.parquet', engine='pyarrow', index=False)
fun(df_geo, col).to_parquet(f'./Data/geo_emb_select_1_v3.parquet', engine='pyarrow', index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
fun(df_trx, col)

  0%|          | 0/3 [00:00<?, ?it/s]

,mon,target_1,target_2,target_3,target_4,client_id,year,report_next_end,emb_trx,event_time,...,euclidean_pos_target_4,euclidean_neg_target_4,cityblock_pos_target_1,cityblock_neg_target_1,cityblock_pos_target_2,cityblock_neg_target_2,cityblock_pos_target_3,cityblock_neg_target_3,cityblock_pos_target_4,cityblock_neg_target_4
0,4,0,0,0,0,1d55174bce3ef488233380aca206ca2fb51661b1c46495...,2022,2022-05-31,"[-1.0, -0.99988127, -1.0, -0.1718208, -0.03291...",2022-04-30,...,4.363934,4.412406,21.745615,21.644463,21.716425,22.425014,21.786354,21.461103,21.698631,21.983650
1,5,1,0,0,0,1d5d052f87d6bd22a30b5df160fffdcdd11014feffe75c...,2022,2022-06-30,"[-1.0, -0.21202722, 0.31116307, -0.1718208, -0...",2022-05-30,...,4.527299,4.571558,22.464161,22.018253,22.368736,23.462372,22.500618,21.856731,22.375908,22.538618
2,5,0,0,0,0,1d68b588164639d64879b33da867818ea7401b6eb7caff...,2022,2022-06-30,"[-1.0, -0.21202722, -1.0, -0.1718208, -1.0, 1....",2022-05-30,...,4.454448,4.527771,22.047600,21.787582,21.999426,22.430082,22.123806,21.434460,21.950443,22.465048
3,7,0,0,0,0,1d817e82e1cc594e32e10217b716faa3ad079261975097...,2022,2022-08-31,"[1.0, 1.0, -1.0, 1.0, -0.03291097, -1.0, -0.48...",2022-07-31,...,4.183972,4.127052,19.940399,20.139122,19.968168,20.235405,19.916046,20.268106,19.991022,19.812943
4,5,1,0,0,0,1daf5849cf68892983c584c8ee97b8329307584a834e79...,2022,2022-06-30,"[1.0, -0.98695296, 1.0, 1.0, 1.0, -1.0, -0.481...",2022-05-30,...,4.394702,4.302628,21.656155,21.736391,21.654423,22.407677,21.616524,21.947964,21.724524,21.211716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406137,6,0,0,0,0,ff0c295f5b534ca6c2d47ad91d0fbe97644f42de9df873...,2022,2022-07-31,"[1.0, 1.0, -1.0, 1.0, -0.03291097, 1.0, 1.0, 1...",2022-06-30,...,4.531089,4.712782,22.882238,23.221979,22.972759,21.461277,22.876514,23.233818,22.819944,23.888477
406138,8,0,0,0,0,ff4a98bcab383f2117cbb91c0d9eb95a5008de0367b1dd...,2022,2022-09-30,"[-1.0, -0.21202722, -1.0, -0.1718208, -0.03291...",2022-08-30,...,4.768586,4.870746,24.032616,23.671265,23.964836,24.552662,24.132126,23.189316,23.900486,24.588673
406139,8,0,0,0,0,ff7cc72f879ec6b041fa7f23ff91f23362c5c9b0ad9b3a...,2022,2022-09-30,"[-1.0, -0.21202722, 1.0, -0.1718208, -0.032910...",2022-08-30,...,3.962089,3.926380,18.957394,18.668785,18.901518,19.358107,18.924635,18.863594,18.940819,18.674974
406140,3,0,0,0,0,ffc09c3decd0f53763e4737accf686bade81f1fc86da94...,2022,2022-04-30,"[-1.0, -0.21202722, 1.0, -1.0, -1.0, -1.0, -1....",2022-03-30,...,4.772995,4.790474,24.384068,24.211756,24.360424,24.335340,24.360119,24.340271,24.338057,24.511551
